# Diabetes Prediction
### Mar 2021
* Instructor: Arnab Bose
* Author: Elly Yang

## Feature Selection

In [1]:
import numpy as np
import pandas as pd

def select(df):
    # join with SyncTranscript
    ## nTranscript (number of transcripts)
    Transcript = pd.read_csv('SyncTranscript.csv')
    df = df.merge(Transcript.groupby(by='PatientGuid').size().to_frame('nTranscript'), on='PatientGuid', how='left')
    ## Height, Weight, BMI, SystolicBP, DiastolicBP, RespiratoryRate
    TranscriptList = ['Height', 'Weight', 'BMI', 'SystolicBP', 'DiastolicBP', 'RespiratoryRate']
    for i in TranscriptList: Transcript.loc[Transcript[i].isnull(), i] = Transcript.loc[~Transcript[i].isnull(), i].mean()
    df = df.merge(Transcript.groupby(by='PatientGuid')[TranscriptList].mean(), on='PatientGuid', how='left')
    ## nTranscriptPS (number of top 5 frequent PhysicianSpecialty)
    TranscriptPS = list(Transcript['PhysicianSpecialty'].value_counts()[:5].index)
    for i in TranscriptPS:
        n = Transcript[Transcript['PhysicianSpecialty'] == i].groupby(by='PatientGuid').size()
        df = df.merge(n.to_frame('nTranscriptPS'+str(TranscriptPS.index(i))), on='PatientGuid', how='left')
    
    # join with SyncSmokingStatus
    ## nCigs (number of cigarettes) converted from nominal to numeric
    SmokingStatus = pd.read_csv('SyncSmokingStatus.csv')
    SmokingStatus['nCig'] = 0
    nCigs = ['Few (1-3) cigarettes per day', 'Up to 1 pack per day', '1-2 packs per day', '2 or more packs per day']
    for i in nCigs: SmokingStatus.loc[SmokingStatus['Description'] == i, 'nCig'] = nCigs.index(i)+1

    # join with SyncPatientSmokingStatus
    PatientSmokingStatus = pd.read_csv('SyncPatientSmokingStatus.csv')
    PatientSmokingStatus = PatientSmokingStatus.merge(SmokingStatus, on='SmokingStatusGuid', how='left')
    df = df.merge(PatientSmokingStatus.groupby(by='PatientGuid').mean()['nCig'].to_frame('avgCig'), on='PatientGuid', how='left')
    
    # join with SyncLabResult
    ## nLabResult (number of LabResult)
    LabResult = pd.read_csv('SyncLabResult.csv')
    df = df.merge(LabResult.groupby(by='PatientGuid').size().to_frame('nLabResult'), on='PatientGuid', how='left')
    
    # join with SyncLabPanel
    ## nLabPanel (number of LabPanel)
    LabPanel = pd.read_csv('SyncLabPanel.csv')
    LabPanel = LabPanel.merge(LabResult[['LabResultGuid', 'PatientGuid']])
    df = df.merge(LabPanel.groupby(by='PatientGuid').size().to_frame('nLabPanel'), on='PatientGuid', how='left')
    ## nLabPanelPN (number of top 5 frequent PanelName)
    LabPanelPN = list(LabPanel['PanelName'].value_counts()[:5].index)
    for i in LabPanelPN:
        n = LabPanel[LabPanel['PanelName'] == i].groupby(by='PatientGuid').size()
        df = df.merge(n.to_frame('nLabPanelPN'+str(LabPanelPN.index(i))), on='PatientGuid', how='left')

    # join with SyncLabObservation
    ## nLabObs (number of LabObs)
    LabObs = pd.read_csv('SyncLabObservation.csv')
    LabObs = LabObs.merge(LabPanel[['LabPanelGuid', 'PatientGuid']])
    df = df.merge(LabObs.groupby(by='PatientGuid').size().to_frame('nLabObs'), on='PatientGuid', how='left')
    ## LabObsHL (number of top 5 frequent HL7Text)
    LabObsHL = list(LabObs['HL7Text'].value_counts()[:5].index)
    for i in LabObsHL:
        n = LabObs[LabObs['HL7Text'] == i].groupby(by='PatientGuid').size()
        df = df.merge(n.to_frame('nLabObsHL'+str(LabObsHL.index(i))), on='PatientGuid', how='left')
    
    # join with SyncDiagnosis
    ## nAcute: number of Acute
    Diagnosis = pd.read_csv('SyncDiagnosis.csv')
    df = df.merge(Diagnosis.groupby(by='PatientGuid').size().to_frame('nDiagnosis'), on='PatientGuid', how='left')
    df = df.merge(Diagnosis.groupby(by='PatientGuid').sum()['Acute'].to_frame('nAcute'), on='PatientGuid', how='left')
    ## ICD9 codes: https://en.wikipedia.org/wiki/List_of_ICD-9_codes
    Diagnosis['ICD9Num'] = Diagnosis.loc[Diagnosis['ICD9Code'].str[0].str.isdigit(), 'ICD9Code'].astype(float)
    Diagnosis['ICD9Group'] = pd.cut(Diagnosis['ICD9Num'], 
                                    [1.0, 140.0, 240.0, 280.0, 290.0, 320.0, 390.0, 460.0, 520.0, 580.0, 630.0, 680.0, 710.0, 740.0, 760.0, 780.0, 800.0, 1000.0], 
                                    labels=['icd9_' + str(i+1) for i in range(17)], 
                                    include_lowest=True).astype(str)
    Diagnosis.loc[Diagnosis['ICD9Group'].isna(), 'ICD9Group'] = 'icd9_0'
    ## ICD9 codes 240–279: endocrine, nutritional and metabolic diseases, and immunity disorders
    Diagnosis.loc[Diagnosis['ICD9Group'] == 'icd9_3', 'ICD9Group'] = 'icd9_3.0'
    Diagnosis['ICD9Group'] = pd.cut(Diagnosis['ICD9Num'],
                                    [240, 249, 260, 270, 279],
                                    labels=['icd9_3.' + str(i+1) for i in range(4)], 
                                    include_lowest=True).astype(str).replace('nan', np.NaN).fillna(Diagnosis['ICD9Group'])
    ## ICD9 codes 390–459: diseases of the circulatory system
    Diagnosis.loc[Diagnosis['ICD9Group'] == 'icd9_8', 'ICD9Group'] = 'icd9_8.0'
    Diagnosis['ICD9Group'] = pd.cut(Diagnosis['ICD9Num'],
                                    [390, 393, 401, 410, 415, 420, 430, 440, 451, 459],
                                    labels=['icd9_8.' + str(i+1) for i in range(9)], 
                                    include_lowest=True).astype(str).replace('nan', np.NaN).fillna(Diagnosis['ICD9Group'])
    ## top 5 frequent ICD9 codes 270–279: Other metabolic and immunity disorders
    for i in Diagnosis.loc[Diagnosis['ICD9Group'] == 'icd9_3.4', 'ICD9Num'].value_counts().head(5).index: 
        Diagnosis.loc[Diagnosis['ICD9Num'] == i, 'ICD9Group'] = 'icd9_3.4_' + str(i)
    ## top 5 frequent ICD9 codes 410–414: Ischemic heart disease
    for i in Diagnosis.loc[Diagnosis['ICD9Group'] == 'icd9_8.3', 'ICD9Num'].value_counts().head(5).index: 
        Diagnosis.loc[Diagnosis['ICD9Num'] == i, 'ICD9Group'] = 'icd9_8.3_' + str(i)
    df = df.merge(pd.crosstab(index=Diagnosis['PatientGuid'], columns=Diagnosis['ICD9Group']), on='PatientGuid', how='left')
    
    # join with SyncMedication
    ## nMedication: number of Medication
    Medication = pd.read_csv('SyncMedication.csv')
    df = df.merge(Medication.groupby(by='PatientGuid').size().to_frame('nMedication'), on='PatientGuid', how='left')
    ## nMedicationMN (number of top 5 frequent MedicationName)
    MedicationMN = list(Medication['MedicationName'].value_counts()[:5].index)
    for i in MedicationMN:
        n = Medication[Medication['MedicationName'] == i].groupby(by='PatientGuid').size()
        df = df.merge(n.to_frame('nMedicationMN'+str(MedicationMN.index(i))), on='PatientGuid', how='left')
    ## Med_Diag: medication prescribed based on ICD9
    Med_Diag = pd.merge(Medication, Diagnosis, on=['PatientGuid', 'DiagnosisGuid'])
    Med_Diag_ct = pd.crosstab(index=Med_Diag['PatientGuid'], columns=Med_Diag['ICD9Group'])
    Med_Diag_ct.columns = ['med_'+i for i in Med_Diag_ct.columns]
    df = df.merge(Med_Diag_ct, on='PatientGuid', how='left')
    
    df = df.fillna(0)
    return(df)

## Load Data

In [2]:
from sklearn.model_selection import train_test_split
Patient = pd.read_csv('SyncPatient.csv')
Patient = select(Patient).drop(['PatientGuid'], axis=1)
X = Patient.drop(['DMIndicator'], axis=1)
y = Patient['DMIndicator']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=.1)

## Evaluate Model Performance with [Brier Score](https://en.wikipedia.org/wiki/Brier_score)

In [3]:
from sklearn.metrics import brier_score_loss
def brier_score(model): 
    print(model.__class__.__name__, 
          'brier score: %.3f' %brier_score_loss(y_val, model.predict_proba(X_val)[:, 1]))

## Random Forest

In [4]:
from sklearn.model_selection import GridSearchCV
def grid_search(model, params):
    grid_search = GridSearchCV(model, params, cv=3, scoring='neg_brier_score')
    grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_

In [5]:
from sklearn.ensemble import RandomForestClassifier
rf = grid_search(RandomForestClassifier(),
                      {'n_estimators': [1000, 2000, 4000], 'min_samples_split': np.linspace(.1, .5, 5)})
brier_score(rf)

RandomForestClassifier brier score: 0.124


## Gradient Boosting

In [6]:
# early stopping (or use staged_predict)
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(warm_start=True)
min_score = np.inf
score_increase = 0
for i in range(1,100):
    gb.n_estimators = i
    gb.fit(X_train, y_train)
    score = brier_score_loss(y_train, gb.predict_proba(X_train)[:,1])
    if score < min_score:
        min_score = score
        score_increase = 0
    else:
        score_increase += 1
        if score_increase == 5: break
brier_score(gb)

GradientBoostingClassifier brier score: 0.112


## Soft Voting

In [7]:
from sklearn.ensemble import VotingClassifier
clf = VotingClassifier(estimators=[('random forest', rf), ('gradient boosting',gb)], 
                       voting='soft')
clf.fit(X_train, y_train)
brier_score(clf)

VotingClassifier brier score: 0.115
